In [ ]:
## needs ~50 gb memory
import pandas as pd 
import numpy as np 

import gc

members = pd.read_csv('../input/members.csv') # more member msno than training examples.  some train msno are not in members.
transactions = pd.read_csv('../input/transactions.csv') 
user_logs = pd.read_csv('../input/user_logs.csv', usecols=['msno','date','total_secs']) # some msno from train are missing.
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/sample_submission_zero.csv')

group = transactions.groupby('msno')
trans_summ_stats = group.agg({'msno' : { 'total_order' : 'count' },
                        'plan_list_price' : {'plan_net_worth' : 'sum'},
                         'actual_amount_paid' : {'mean_payment_each_transaction' : 'mean',
                                                 'total_actual_payment' : 'sum'},
                         'is_cancel' : {'cancel_times' : lambda x : sum(x==1)}})


trans_summ_stats.columns = trans_summ_stats.columns.droplevel(0)
trans_summ_stats.reset_index(inplace=True)

base = pd.merge(left=members, right=trans_summ_stats, how='right', on='msno')
print("1",base.columns)
# add summary stats for the user logs

group = user_logs.groupby('msno')
logs_summ_stats = group.agg({
    'msno': {'total_listens' : 'count'},
    'date': {'last_listen_date' : 'max'}
    })

logs_summ_stats.columns = logs_summ_stats.columns.droplevel(0)
logs_summ_stats.reset_index(inplace=True)

base = base.merge(logs_summ_stats, how='left', on='msno')

# the last time a user played a song - the date the user registered
base['dist_last_listen_registration'] = base['last_listen_date'] - base['registration_init_time'] 
# last listen time - expiration date
base['dist_expiration_last_listen'] = base['expiration_date'] - base['last_listen_date']
base['listens_per_day'] = base['total_listens'] / base['dist_last_listen_registration']

base['gender'].fillna('unknown_gender', inplace=True)
dummies = pd.get_dummies(base['gender'])

base.drop('gender', axis=1, inplace=True)
base = base.join(dummies)
train = train.merge(base, how='left', on='msno')

test = test.merge(base, how='left', on='msno')

train.to_csv('../input/merged-train.csv',index=False)
test.to_csv('../input/merged-test.csv',index=False)

In [ ]:
#print(len(train),len(members), len(test))

#train.isnull().sum()

In [ ]:
#test

### some msno data is missing..
#transactions[transactions['msno']=='4HykJ6KaHOBgdGIa4mlxU9bxFRc+Co/PGcrhgTftOm0=']  ## has records
#user_logs[user_logs['msno']=='4HykJ6KaHOBgdGIa4mlxU9bxFRc+Co/PGcrhgTftOm0='] ## no records
#members[members['msno']=='4HykJ6KaHOBgdGIa4mlxU9bxFRc+Co/PGcrhgTftOm0=']  ## no records
#train[train['msno']=='4HykJ6KaHOBgdGIa4mlxU9bxFRc+Co/PGcrhgTftOm0=']  ## has records

#train.isnull().sum()